Procesamiento Refined

In [98]:
#Importamos Pandas y cargamos archivos
import pandas as pd

# Cargamos el archivo de medallas
df_medals = pd.read_csv('./datos/raw/olympic_medals.csv')

# Cargamos el archivo de resultados
df_results = pd.read_csv('./datos/raw/olympic_results_from_pkl.csv')

# Cargamos el archivo de atletas
df_atletas = pd.read_csv('./datos/raw/olympic_athletes.csv')

Definimos Funciones a Utilizar

In [66]:
#funcion para sutituir un valor nulo de una columna por un valor existente de la misma columna buscando por otro codigo relacionado
def sustituir_valor_filtrado(df, value, filter):
    # Identificar las filas que tienen valor nulo en la columna 'value'
    mask = pd.isnull(df[value])    
    # Filtrar el DataFrame original para obtener solo las filas donde 'value' es nulo
    df_nulls = df[mask]  
    # Obtener los valores únicos de 'filter' para iterar sobre ellos
    unique_filters = df_nulls[filter].unique() 
    # Iterar sobre cada valor único de 'filter' y rellenar los valores nulos correspondientes
    for f in unique_filters:
        # Filtrar las filas donde 'filter' coincide y 'value' no es nulo
        mask_filter = (df[filter] == f) & pd.notna(df[value])
        df_filter = df[mask_filter]
        
        # Si hay filas en df_filter, asignar el valor de la primera fila a las filas correspondientes en df_nulls
        if not df_filter.empty:
            first_value = df_filter.iloc[0][value]
            df.loc[mask & (df[filter] == f), value] = first_value
    
    return df


def llenar_url_faltante(df_rellenar, df_atletas):
    # Crear un diccionario de nombres de atletas a URLs para evitar múltiples accesos a df_atletas
    nombre_a_url = df_atletas.set_index('athlete_full_name')['athlete_url'].to_dict()
    
    # Función para obtener la URL o cadena vacía si no se encuentra
    def obtener_url(nombre):
        return nombre_a_url.get(nombre, '')

    # Máscara de filas donde athlete_url es nulo en df_rellenar
    mask = df_rellenar['athlete_url'].isnull()

    # Obtener los nombres únicos de atletas donde athlete_url es nulo
    nombres_a_buscar = df_rellenar.loc[mask, 'athlete_full_name'].unique()

    # Llenar los valores nulos en df_rellenar con las URLs encontradas en df_atletas
    df_rellenar.loc[mask, 'athlete_url'] = df_rellenar.loc[mask, 'athlete_full_name'].apply(obtener_url)
    
    return df_rellenar

Limpieza Dataset Medallas

In [110]:
#Remplazamos columna country_cod nulas por columna country_cod no nulas cuando country_3_letter_code es igual en ambas filas
df_medals_cc = sustituir_valor_filtrado(df_medals, 'country_code', 'country_3_letter_code')

#Si aun existen datos en la columna country_code nulos les asignamos el valor de la columna country_3_letter_code
df_medals_cc['country_code'].fillna(df_medals_cc['country_3_letter_code'], inplace = True)

# Ya que en muchas de las filas la columna participant_title es igual a la columna country_name, sustituimos participant_title nulos por country_name

#Continuamos a partir del dataframe con la columna country_code sin valores nulos
df_medals_pt = df_medals_cc
df_medals_pt['participant_title'].fillna(df_medals_pt['country_name'], inplace=True)


#Ya que la url es un valor fijo + el nombre del atleta, cuando la url del atleta es nula y el nombre no lo es, 
#parseamos el nombre para darle un formato especifico y lo concatenamos al valor fijo de la url
#Continuamos a partir del dataframe con la columna country_code y participant_title sin valores nulos
df_medals_au = llenar_url_faltante(df_medals_pt, df_atletas)

#Por ultimo rellenamos los valores nulos faltantes con el dato vacio ''
df_clean_medals = df_medals_au.fillna('')

display(df_clean_medals.isnull().sum())

discipline_title         0
slug_game                0
event_title              0
event_gender             0
medal_type               0
participant_type         0
participant_title        0
athlete_url              0
athlete_full_name        0
country_name             0
country_code             0
country_3_letter_code    0
dtype: int64

Limpieza Dataset Resultados

In [109]:
#Completamos los las filas sin medallas con 'NA'
df_results_mt = df_results
df_results_mt['medal_type'].fillna('NA', inplace = True)

#Completamos los las filas sin atletas con ''
df_results_a = df_results_mt
df_results_a['athletes'].fillna('', inplace = True)

#Ya que la url es un valor fijo + el nombre del atleta, cuando es un atleta, la url es nula y el nombre no lo es, 
#parseamos el nombre para darle un formato especifico y lo concatenamos al valor fijo de la url
df_results_url = llenar_url_faltante(df_results_a, df_atletas)

#Si es un equipo y el atleta y la url son nulas sustituimos por ''
df_results_ua = df_results_a

condicion = df_results_ua['participant_type'] == 'GameTeam'
df_results_ua.loc[condicion, ['athlete_url', 'athlete_full_name']] = df_results_ua.loc[condicion, ['athlete_url', 'athlete_full_name']].fillna('')


#Sustituimos rank_equal por False cuando es nulo 
df_results_re = df_results_ua
df_results_re['rank_equal'].fillna(False, inplace = True)

#Sustituimos rank_position por NM(No Mark) cuando es nulo 
df_results_rp = df_results_re
df_results_rp['rank_position'].fillna('NM', inplace = True)

#Remplazamos columna country_cod nulas por columna country_cod no nulas cuando country_3_letter_code es igual en ambas filas
df_results_cc = sustituir_valor_filtrado(df_results_rp, 'country_code', 'country_3_letter_code')

#Si aun existen datos en la columna country_code nulos les asignamos el valor de la columna country_3_letter_code
df_results_cc['country_code'].fillna(df_results_cc['country_3_letter_code'], inplace = True)

#Remplazamos columna value_type nulas por columna value_type no nulas cuando discipline_title es igual en ambas filas
df_results_vt = sustituir_valor_filtrado(df_results_cc, 'value_type', 'discipline_title')

#Si aun existen datos en la columna value_type nulos les asignamos el valor NT(No Type)
df_results_vt['value_type'].fillna('NT', inplace = True)

#Sustituimos value_unit por NV(No Value) cuando es nulo 
df_results_vu = df_results_vt
df_results_vu['value_unit'].fillna('NV', inplace = True)

#Por ultimo rellenamos los valores nulos faltantes con el dato vacio '' y eliminamos la columna Unnamed: 0
df_clean_results = df_results_vu.fillna('')
df_clean_results = df_clean_results.drop(columns=['Unnamed: 0'])

display(df_clean_results.isnull().sum())

discipline_title         0
event_title              0
slug_game                0
participant_type         0
medal_type               0
athletes                 0
rank_equal               0
rank_position            0
country_name             0
country_code             0
country_3_letter_code    0
athlete_url              0
athlete_full_name        0
value_unit               0
value_type               0
dtype: int64

Limpieza Dataset Atletas

In [108]:
df_atletas_fg = df_atletas

df_fg_null = df_atletas_fg[pd.isnull(df_atletas_fg['first_game'])] 
for i in range(len(df_fg_null)):
    fila = df_fg_null.iloc[i]
    # se filtra df_clean_results por athlete_full_name
    df_res_filt = df_clean_results[df_clean_results['athlete_full_name'] == fila['athlete_full_name']]    
    # Se divide slug_game y se asigna a nuevas columnas
    df_res_filt.loc[:, ['nom', 'anio']] = df_res_filt['slug_game'].str.split('-', expand=True)
    # Se filtra df_res_filt por el año mínimo
    df_filt = df_res_filt[df_res_filt['anio'] == df_res_filt['anio'].min()]  
    # Si se encontraron 1 o mas resutlados para el atleta de fila, se asigna el primer valor slug_game del nuevo dataframe filtrado
    if df_filt['slug_game'].count() > 0:
        df_atletas_fg.at[df_atletas_fg[df_atletas_fg['athlete_full_name'] == fila['athlete_full_name']].index[0], 'first_game'] = df_filt['slug_game'].iloc[0]

#Se rellena la columna con 'NG'(No Game) cuando no se encontraron resultados para el atleta
df_atletas_fg['first_game'].fillna('NG', inplace = True)

#Se completa fecha de nacimiento con promedio de nacimientos para su primer juego
df_atletas_ab = df_atletas_fg

df_ab_null = df_atletas_ab[pd.isnull(df_atletas_fg['athlete_year_birth'])] 
df_ab_nna = df_atletas_ab[pd.notna(df_atletas_fg['athlete_year_birth'])] 
for i in range(len(df_ab_null)):
    fila = df_ab_null.iloc[i]
    df_atl_filt = df_ab_nna[df_ab_nna['first_game'] == fila['first_game']]
    prom = df_atl_filt['athlete_year_birth'].mean()
    df_atletas_ab.at[df_atletas_ab[df_atletas_ab['athlete_full_name'] == fila['athlete_full_name']].index[0], 'athlete_year_birth'] = prom

#Se rellena la columna con 0 cuando no se encontraron un pormedio para el atleta
df_atletas_fg['athlete_year_birth'].fillna(0, inplace = True)

#Se rellena columna bio con '' cuando es nulo
df_atletas_bi = df_atletas_fg
df_atletas_bi['bio'].fillna('', inplace = True)


#Se rellenan valores de athlete_medals con la cantidad de medallas que se cuentan en df_clean_medals y se convierte olumna a entero 
#para borrar saltos de linea y datos innecesarios
df_atletas_me = df_atletas_bi
# Filtrar filas donde athlete_medals es nulo
df_me_null = df_atletas_me[pd.isnull(df_atletas_me['athlete_medals'])]
# Hacer un merge entre df_atletas_bi y df_clean_medals para obtener las medallas por atleta
merged = pd.merge(df_atletas_me, df_clean_medals, on='athlete_full_name', how='left')
# Calcular el total de medallas por atleta
medals_count = merged.groupby('athlete_full_name').size().reset_index(name='total_medals')
# Actualizar df_atletas_bi con los totales calculados
df_atletas_me = pd.merge(df_atletas_me, medals_count, on='athlete_full_name', how='left')
df_atletas_me['athlete_medals'] = df_atletas_me['total_medals']
df_atletas_me.drop(columns=['total_medals'], inplace=True)

display(df_atletas_me.isnull().sum())

athlete_url             0
athlete_full_name       0
games_participations    0
first_game              0
athlete_year_birth      0
athlete_medals          0
bio                     0
dtype: int64